In [1]:
DATA_PATH = '../../data/CRTS2/'

In [2]:
import pandas as pd
import sys
import numpy as np
import astropy.time as astime

In [3]:
sys.path.append("../..")
import measurements

Import transient lightcurves

In [4]:
filename = 'transient_lightcurves.pickle'
indir = DATA_PATH; filepath = indir + filename
df_tra = pd.read_pickle(filepath)
df_tra.shape

(451474, 4)

Filter transient lightcurves

In [5]:
# Delete rows of blended observations
df_tra = df_tra.drop_duplicates(['TransientID','MJD'], keep='first')

In [6]:
# Add observation count to every transient
df_count = df_tra.groupby('TransientID', as_index=False).count()
df_count['ObsCount'] = df_count['Mag']
df_count = df_count[['TransientID', 'ObsCount']]
df_tra = df_tra.merge(df_count, how='inner')

In [7]:
# Remove objects with less than 5 observations
df_tra = df_tra[df_tra.ObsCount >= 5]

Import permanent lightcurves

In [8]:
filename = 'permanent_lightcurves.pickle'
indir = DATA_PATH; filepath = indir + filename
df_per = pd.read_pickle(filepath)
df_per.shape

(1924409, 4)

In [9]:
# Delete rows of blended observations
df_per = df_per.drop_duplicates(['ID','MJD'], keep='first')
df_per.shape

(1802695, 4)

In [10]:
# Add observation count to every permanent
df_count = df_per.groupby('ID', as_index=False).count()
df_count['ObsCount'] = df_count['Mag']
df_count = df_count[['ID', 'ObsCount']]
df_per = df_per.merge(df_count, how='inner')

In [11]:
# Remove objects with less than 5 observations
df_per = df_per[df_per.ObsCount >= 5]
df_per.shape

(1798465, 5)

In [12]:
df_per.ID.unique().shape

(15193,)

In [13]:
# Sample subset of same size as transients
sample_size = df_tra.TransientID.unique().shape[0]
IDs = np.random.choice(df_per.ID.unique(), size=sample_size, replace=False)
df_per = df_per[df_per.ID.isin(IDs)]
df_per.shape

(512281, 5)

 Define extract features functionality

In [79]:
def stetson_k(data):
  '''
  Welch-Stetson variability index K (Stetson 1996)
  '''
  n = len(data[1])
  stetson_k = 0
  if n > 1:
    if len(data) > 2:
      delta = np.sqrt(float(n) / (n - 1)) * (data[1] - np.mean(data[1])) / data[2] 
    else:
      delta = np.sqrt(float(n) / (n - 1)) * (data[1] - np.mean(data[1]))
    top = abs(delta).sum() / n
    bottom = np.sqrt((delta * delta).sum() / n)
    stetson_k = top / bottom
  return stetson_k

In [80]:
def extract_features(df, feature_dict):
    df = df.copy()
    df['Flux'] = measurements.__mag_to_flux__(df.Mag)
    df['Date'] = astime.Time(df.MJD, format='mjd').datetime
    df = df.sort_values('Date')
#    print(df.Mag.as_matrix(), df.Magerr.as_matrix())
    data = df[['Date', 'Mag', 'Magerr']].transpose().as_matrix()
    feature_dict['stetson_j'].append(stetson_j(data))
#    feature_dict['2stetson_k'].append(stetson_k(data))
    feature_dict['stetson_j_my'].append(my_stetson_j(df.Mag, df.Magerr, df.Date, True))
#    feature_dict['stetson_k'].append(measurements.stetson_k(df.Mag, df.Magerr))
    #feature_dict['stetson_k'].append(measurements.stetson_k(df.Mag, df.Magerr))

In [162]:
def __stetson_sigmas__(ss_mag, ss_magerr):
    '''
    Calculates the relative errors (sigmas) for stetson measurements.
    '''
    n = ss_mag.shape[0]
    sigmas = np.sqrt(float(n) / (n - 1)) * (ss_mag - ss_mag.mean()) / ss_magerr
    return sigmas

def __datetime_diff_to_int_timedelta__(ss_datetime_diff):
    '''
    Convert datetime series to integer timedelta
    '''
    return ss_datetime_diff.dt.total_seconds() / 3600
def __datetime_diff_to_int_timedelta2__(diff):
    '''
    Convert datetime series to integer timedelta
    '''
    return diff.total_seconds() / 3600

In [279]:
import numpy
def stetson_j(data):
    '''
    Welch-Stetson variability index J (Stetson 1996) with weighting scheme from (Zhang et al. 2003)
    taking successive pairs in time-order
    '''
    n = len(data[1])
    stetson_j = 0
    if n > 1:
        if len(data) > 2: 
            delta = np.sqrt(float(n) / (n - 1)) * (data[1] - np.mean(data[1])) / data[2] 
        else:
            delta = np.sqrt(float(n) / (n - 1)) * (data[1] - np.mean(data[1])) 
        sum = 0.
        w_sum = 0.
        dt = 0.
        my_sum = 0.
        for i in range(n - 1):
            dt += ( (data[0][i + 1] - data[0][i]).total_seconds() / 3600 )
            my_sum += ((data[0][i + 1] - data[0][i]).total_seconds() / 3600 )
            dt = dt / float(n - 1)
            if i == n-2: print(dt)
        print('MY_SUM', my_sum)
        for i in range(n - 1):
            wk = numpy.exp(-(data[0][i + 1] - data[0][i]).total_seconds() / 3600 / dt)
            #if(i == 0): print(wk)
            pk = delta[i] * delta[i + 1]
            sum += wk * np.sign(pk) * np.sqrt(abs(pk))
            w_sum += wk
            stetson_j = sum / w_sum
    return stetson_j

In [280]:
def my_stetson_j(ss_mag, ss_magerr, ss_date, exponential=False):
    '''
    The Welch-Stetson J variability index (Stetson 1996).
    A robust standard deviation.
    Optional exponential weighting scheme (Zhang et al. 2003) taking successive pairs in time order.
    NOTE: ss_flux must be ordered by it's corresponding date in ss_date.
    '''
    n = ss_mag.shape[0]
    if n <= 1: return 0
    # Calculate sigmas: Relative Errors
    sigmas = __stetson_sigmas__(ss_mag, ss_magerr)
    # Calculate weights
    w = np.ones(int(n)); w[0] = 0
    if exponential:
        # Calculate mean dt: delta-time
        dt = __datetime_diff_to_int_timedelta__(ss_date.diff()).sum() / (n-1)
        print('DT2', __datetime_diff_to_int_timedelta__(ss_date.diff()).sum(), dt)
        # Re-calculate Weights
        w = np.exp(-__datetime_diff_to_int_timedelta__(ss_date.diff()) / dt)
#    print(dt)
    print(__datetime_diff_to_int_timedelta__(ss_date.diff()).shape)
    # Calculate p: product of residuals
    p = sigmas * sigmas.shift(1)
    # Return Stetson J measuerement
    return (w * np.sign(p) * p.abs().pow(1./2)).sum() / w.sum()

Extract transient features

In [281]:
LOCATION = 11
feature_dict = {'stetson_j_my': [], 'stetson_j':[]}
               # 'stetson_k': [], '2stetson_k':[]}
for trID in df_tra.TransientID.unique():
    df = df_tra[df_tra.TransientID == trID]
#    feature_dict['ID'].append(trID)
    extract_features(df, feature_dict)
    print(feature_dict)
    break
#df_feat_tran = pd.DataFrame(feature_dict)

488.95619340727063
MY_SUM 66581.0811852839
DT2 66581.0811853 5548.42343211
(13,)
{'stetson_j': [-0.76384204983677106], 'stetson_j_my': [-0.7675000063263111]}
